# Pad per il riconoscimento di numeri in Real-time
Questo notebook espone una libreria per testare il nostro modello che abbiamo precedentemente addestrato, tramite un pad dove è possibile scrivere dei numeri a mano che successivamente la rete artificiale neurale dovrà riconoscere.
I numeri scritti a mano vengono elaborati e trasformati in immagini 28x28x1. Quindi viene applicato un filtro gaussiano e antialiasing per ridurre l'immagine nel formato adatto.

### Funzionamento
1. Esegui tutti i codici per poter utilizzare il pad
2. Digita solamente una cifra
3. Fai clear per pulire il pad

In [6]:

import tkinter as tk
from PIL import Image, ImageDraw, ImageOps, ImageFilter
import numpy as np
from tensorflow.keras.models import load_model

# caricamento del modello preaddestratoq
model = load_model('best_model.keras')

# Elaborazione dell'immagine per renderla compatibile con il modello
def preprocess_image(image):
    # conversione dell'immagine in scala di grigi e invertita
    grayscale_image = ImageOps.invert(image.convert('L'))
    
    # applicazione del filtro gaussiano
    blurred_image = grayscale_image.filter(ImageFilter.GaussianBlur(1))
    
    # riduzione graduale delle dimensioni dell'immagine e applicazione del filtro Lanczos
    resized_image = blurred_image.resize((140, 140), Image.Resampling.LANCZOS)
    resized_image = resized_image.resize((28, 28), Image.Resampling.LANCZOS)
    
    # Applicazione del thresholding per convertire l'immagine in bianco e nero
    # threshold_image = resized_image.point(lambda x: 0 if x < 128 else 255, '1')
    
    # Normalizzazione dei valori dei pixel
    image_array = np.array(resized_image) / 255.0
    
    return image_array.reshape(1, 28, 28, 1)

# Funzione per la predizione del numero scritto
def predict_digit(image):
    preprocessed_image = preprocess_image(image)
    prediction = model.predict(preprocessed_image)
    probabilities = prediction[0]  # Estrazione delle probabilità di appartenenza ad ogni classe
    predicted_digit = np.argmax(probabilities)  # Numero predetto
    return predicted_digit, probabilities


In [7]:

# Disegno del pad per la scrittura
class DrawingPad:
    def __init__(self, root):
        self.root = root
        self.canvas = tk.Canvas(root, width=280, height=280, bg="white")
        self.canvas.pack()

        self.button_predict = tk.Button(root, text="Fai la predizione", command=self.make_prediction)
        self.button_predict.pack()

        self.button_clear = tk.Button(root, text="Cancella", command=self.clear_canvas)
        self.button_clear.pack()

        self.image = Image.new("RGB", (280, 280), "white")
        self.draw = ImageDraw.Draw(self.image)
        self.canvas.bind("<B1-Motion>", self.paint)

        self.label_result = tk.Label(root, text="Predizione: ", font=("Helvetica", 16))
        self.label_result.pack()

        self.label_probabilities = tk.Label(root, text="", font=("Helvetica", 12))
        self.label_probabilities.pack()
    
    def paint(self, event):
        brush_size = 15  # Dimensione del pennello (diametro in pixel)
        x1, y1 = event.x - brush_size // 2, event.y - brush_size // 2
        x2, y2 = event.x + brush_size // 2, event.y + brush_size // 2
        self.canvas.create_oval(x1, y1, x2, y2, fill="black", outline="black")
        self.draw.ellipse([x1, y1, x2, y2], fill="black")


    def make_prediction(self):
        digit, probabilities = predict_digit(self.image)
        self.label_result.config(text=f"Numero predetto: {digit}")
        formatted_probs = "".join([f"{i}: {prob:.2%} " for i, prob in enumerate(probabilities)])
        self.label_probabilities.config(text=f"Probabilità: {formatted_probs}")

    def clear_canvas(self):
        self.canvas.delete("all")
        self.image = Image.new("RGB", (280, 280), "white")
        self.draw = ImageDraw.Draw(self.image)

root = tk.Tk()
app = DrawingPad(root)
root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
